IAMSI -- 2022-2023
--------


# TME 07: Règles d'association

<font color="RED" size="+1">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms**

Léo PORTE

## Présentation

### But de ce TME

Ce TME a pour but de réaliser une implémentation *intuitive* de l'algorithme **apriori** afin de la comparer à une version efficace de cet algorithme, puis à une implémentation de l'algorithme **fp-growth**.


#### Contenu de l'archive téléchargée

Une fois détarée, l'archive **tme-07.tgz** crée le répertoire <code>tme09</code> contenant :
- un répertoire <code>dataset</code> contenant des bases d'exemples pour tester les algorithmes
- deux fichiers exécutables (Linux): <code>apriori32</code> et <code>fpgrowth32</code> (ces 2 programmes ont été développés par Christian Borgelt, plus d'infos sur sa page web  https://www.borgelt.net/software.html). Ces programmes seront utilisés en fin de séance. Si vous n'êtes pas sous Linux, vous pouvez trouver une version pour votre OS ici:
    - <code>apriori</code> : https://borgelt.net/apriori.html
    - <code>fpgrowth</code> : https://borgelt.net/fpgrowth.html



#### Compte-rendu de la séance

Le compte-rendu de ce TME se compose de ce notebook complété par les réponses aux questions posées.

Ce compte-rendu est à poster : 
- à l'issue de la séance, un premier envoi doit être **obligatoirement** fait avec ce que vous avez réalisé
- si nécessaire, un complément peut être envoyé **au plus tard avant le début de la prochaine séance**.


<font color="RED">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : *Nom1_Nom2.ipynb* 
- *Nom1* et *Nom2* : noms des membres du binôme
- ne pas compresser ou faire une archive: envoyer le notebook tel quel, éventuellement, si vous avez d'autres fichiers à envoyer, les  joindre au message.

In [1]:
# Vérification de la version de Python utilisée:
import sys
sys.path # le path doit contenir python3.5 ou une version supérieure

['/home/gameselo/Bureau/M1_DAC_2022_2023/IAMSI/TME/tme07/tme-07',
 '/home/gameselo/miniconda3/envs/mapsi/lib/python39.zip',
 '/home/gameselo/miniconda3/envs/mapsi/lib/python3.9',
 '/home/gameselo/miniconda3/envs/mapsi/lib/python3.9/lib-dynload',
 '',
 '/home/gameselo/miniconda3/envs/mapsi/lib/python3.9/site-packages',
 '/home/gameselo/Bureau/M1_DAC_2022_2023/PLDAC/datamaestro/src',
 '/home/gameselo/Bureau/M1_DAC_2022_2023/PLDAC/datamaestro_text/src',
 '/home/gameselo/Bureau/M1_DAC_2022_2023/PLDAC/experimaestro-ir/src',
 '/home/gameselo/miniconda3/envs/mapsi/lib/python3.9/site-packages/experimaestro_ir-0.0.0-py3.9.egg',
 '/home/gameselo/Bureau/M1_DAC_2022_2023/PLDAC/cosplade/splade']

## Traitement d'une base d'apprentissage


### Chargement de la base

On commence par travailler sur la base exemple du fichier <code>exemple-1.txt</code> (fourni dans le répertoire datasets). 

Ce fichier contient une transaction par ligne. Chaque transaction est composée d'un groupe d'items séparés par un espace.

On peut charger en Python ce fichier par la commande suivante (le répertoire datasets doit se trouver dans le répertoire courant) :

In [2]:
import csv
with open('datasets/exemple-1.txt', 'r') as fichier:
    lecteur = csv.reader(fichier, delimiter=' ')
    i = 0
    for ligne in lecteur:
        i += 1
        print('ligne',i,':',ligne)

ligne 1 : ['a', 'b', 'c']
ligne 2 : ['a', 'd', 'e']
ligne 3 : ['b', 'c', 'd']
ligne 4 : ['a', 'b', 'c', 'd']
ligne 5 : ['b', 'c']
ligne 6 : ['a', 'b', 'd']
ligne 7 : ['d', 'e']
ligne 8 : ['a', 'b', 'c', 'd']
ligne 9 : ['c', 'd', 'e']
ligne 10 : ['a', 'b', 'c']


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>chargeBase</code> qui prend en argument un nom de fichier, respectant le format énoncé plus haut, le lit et rend un dictionnaire dont les clés sont les numéros de ligne (une transaction) et les valeurs associées les itemsets correspondants représentés sous forme d'ensembles Python (des <code>set()</code> donc).

Ici, il est plus intéressant de représenter un itemset comme un ensemble Python plutôt que comme une liste. Il est ainsi plus facile de réaliser des comparaisons d'ensembles ou des ajouts d'éléments.

Dans le reste de ce document, on appelle **BASE** un dictionnaire de ce type.

In [4]:
def chargeBase(filename, delimiteur=' '):
    base = dict()
    with open(filename, 'r') as source:
        i = 1
        for line in source:
            values = line.rstrip().split(delimiteur)
            base[i] = set(values)
            i += 1
            
    return base

In [5]:
print("Résultat du chargement de 'exemple-1.txt', on obtient : ")
Base1 = chargeBase('datasets/exemple-1.txt')
Base1

Résultat du chargement de 'exemple-1.txt', on obtient : 


{1: {'a', 'b', 'c'},
 2: {'a', 'd', 'e'},
 3: {'b', 'c', 'd'},
 4: {'a', 'b', 'c', 'd'},
 5: {'b', 'c'},
 6: {'a', 'b', 'd'},
 7: {'d', 'e'},
 8: {'a', 'b', 'c', 'd'},
 9: {'c', 'd', 'e'},
 10: {'a', 'b', 'c'}}

On utilise la variable **Base1** dans la suite pour faire référence à cette base.

## Itemsets et support

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>noms_items</code> qui prend en argument une BASE et rend l'ensemble des items qui composent cette base.

In [6]:
def noms_items(base):
    ens = set()
    for i in base.keys():
        for item in base[i]:
            if item not in ens:
                ens.add(item)
    return ens

In [7]:
print("Pour la BASE précédente :")
noms_items(Base1)

Pour la BASE précédente :


{'a', 'b', 'c', 'd', 'e'}

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>singletons</code> qui prend en argument une BASE et rend la liste des itemsets de taille 1 obtenus à partir de cette base.

*Remarque*: attention, ici on utilise une **liste** pour stocker les itemsets (un itemset est un ensemble Python) car il n'est pas possible en Python de créer des ensembles d'ensembles.


In [8]:
def singletons(base):
    sgt = []
    items = noms_items(base)
    for item in items:
        sgt.append({item})
    return sgt

In [9]:
print("Exemple: singletons(Base1) rend :")
singletons(Base1)

Exemple: singletons(Base1) rend :


[{'d'}, {'e'}, {'a'}, {'b'}, {'c'}]

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>comptage</code> qui, pour une BASE et un itemset donnés, rend le nombre de transactions de BASE qui contiennent cet itemset.

In [10]:
def comptage(base, itemset):
    cpt = 0
    for i in base.keys():
        if itemset.issubset(base[i]):
            cpt += 1
    
    return cpt

In [11]:
print("Comptage de l'itemset {'a','b','c'} dans la base précédente : ")
print("comptage(Base1,{'a','b','c'}) rend la valeur "+str(comptage(Base1,{'a','b','c'})))

Comptage de l'itemset {'a','b','c'} dans la base précédente : 
comptage(Base1,{'a','b','c'}) rend la valeur 4


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>support</code> qui, pour une BASE et un itemset donnés, rend le support de cet itemset dans la BASE.

In [12]:
def support(base, itemset):
    return comptage(base, itemset) / len(base)

In [13]:
print("Support de l'itemset {'a','b','c'} dans la base précédente : " \
      +str(support(Base1,{'a','b','c'})))

Support de l'itemset {'a','b','c'} dans la base précédente : 0.4


## Implémentation de l'algorithme a-priori

Dans cette partie, une implémentation de la partie de construction des itemsets fréquents de l'algorithme a-priori est réalisée. On ne s'intéresse pas dans cette question à la génération des règles d'association (mais cela peut être fait en complément).

Votre programme doit pouvoir s'appliquer aux bases fournies dans le répertoire *datasets* (éventuellement, sur au moins les 10 premiers exemples de la base mushrooms).

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>apriori_gen</code> qui prend en argument une liste d'itemsets de même longueur $k$, applique l'algorithme apriori-gen pour rendre la liste des itemsets candidats de longueurs $k+1$.


In [14]:
def apriori_gen(liste_itemsets):
    if liste_itemsets: 
        k = len(liste_itemsets[0])
    else:
        k = 0
    for itemset in liste_itemsets:
        if len(itemset) != k:
            return "Erreur : les itemsets ne sont pas tous de la même taille"
    
    C = []
    for f1 in liste_itemsets:
        for f2 in liste_itemsets:
            c = f1.union(f2)
            if len(c) == k+1:
                C.append(c)
    
    nodub_C = []
    for c in C:
        if c not in nodub_C:
            keep = True
            for i in c:
                if c - {i} not in liste_itemsets:
                    keep = False
                    break
                    
            if keep:
                nodub_C.append(c)
                
    return nodub_C

In [15]:
print("Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend ")
print(apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]))

print("\nExemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend ")
print(apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]))

Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend 
[{'a', 'b'}, {'a', 'c'}, {'a', 'd'}, {'b', 'c'}, {'d', 'b'}, {'d', 'c'}]

Exemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend 
[{'a', 'd', 'b'}, {'d', 'b', 'c'}]


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>apriori</code> qui prend en argument une BASE et une valeur réelle comprise entre 0 et 1, et qui rend une liste de tuples dont le premier élément et un itemset trouvé et le deuxième élément est la valeur de support correspondante.

In [16]:
def apriori(base, minsup):
    C = [singletons(base)]
    F = []
    k = 0
    while C[k]:
        Fk = []
        for c in C[k]:
            if support(base, c) >= minsup:
                Fk.append(c)
        k += 1
        C.append(apriori_gen(Fk))
        F += Fk
    
    F_supp = []
    for elem in F:
        F_supp.append((elem, support(base, elem)))
    
    return F_supp

In [17]:
print("Exemple: apriori(Base1, 0.3]) rend ")
print(apriori(Base1,0.3))

Exemple: apriori(Base1, 0.3]) rend 
[({'d'}, 0.7), ({'e'}, 0.3), ({'a'}, 0.6), ({'b'}, 0.7), ({'c'}, 0.7), ({'d', 'e'}, 0.3), ({'a', 'd'}, 0.4), ({'d', 'b'}, 0.4), ({'d', 'c'}, 0.4), ({'a', 'b'}, 0.5), ({'a', 'c'}, 0.4), ({'b', 'c'}, 0.6), ({'a', 'd', 'b'}, 0.3), ({'d', 'b', 'c'}, 0.3), ({'a', 'b', 'c'}, 0.4)]


## Expérimentations

### Vérification avec la base du TD

Utiliser la fonction <code>apriori</code> avec la base de transactions du TD afin de vérifier que les itemsets obtenus sont corrects.

In [18]:
BaseTD = chargeBase('datasets/basetd.txt')

In [19]:
print("Exemple: apriori(BaseTD, 0.3) rend ")
print(apriori(BaseTD,0.3))

Exemple: apriori(BaseTD, 0.3) rend 
[({'E'}, 0.6), ({'C'}, 0.5), ({'D'}, 0.9), ({'B'}, 0.7), ({'A'}, 0.5), ({'E', 'D'}, 0.6), ({'E', 'B'}, 0.4), ({'E', 'A'}, 0.4), ({'C', 'D'}, 0.4), ({'C', 'B'}, 0.3), ({'D', 'B'}, 0.6), ({'D', 'A'}, 0.4), ({'A', 'B'}, 0.3), ({'E', 'D', 'B'}, 0.4), ({'E', 'D', 'A'}, 0.4)]


### Comparaisons avec les implémentations apriori et fpgrowth

<font color="RED" size="+1">**[Q]**</font>  Tester votre fonction <code>apriori</code> sur les 3 bases données dans datasets et confronter les résultats obtenus avec ceux fournis par l'exécutable <code>apriori</code>.

In [20]:
print("Résultat du chargement de 'mushrooms.txt', on obtient : ")
Base_mushrooms = chargeBase('datasets/mushrooms.txt',delimiteur=',')
#Base_mushrooms
print(len(Base_mushrooms),' transactions.')

Résultat du chargement de 'mushrooms.txt', on obtient : 
8416  transactions.


In [21]:
print("Pour la BASE précédente :")
print(noms_items(Base_mushrooms))
print("Exemple: singletons(Base_mushrooms) rend :")
print(singletons(Base_mushrooms))

Pour la BASE précédente :
{'ALMOND', 'ENLARGING', 'CREOSOTE', 'GRASSES', 'YELLOW', 'PATHS', 'BELL', 'FLARING', 'CINNAMON', 'CHOCOLATE', 'CROWDED', 'EVANESCENT', 'CLUSTERED', 'RED', 'BRUISES', 'EDIBLE', 'CLUB', 'SUNKEN', 'ATTACHED', 'TAPERING', 'ONE', 'LEAVES', 'FIBROUS', 'EQUAL', 'SPICY', 'WHITE', 'MUSTY', 'TWO', 'ANISE', 'ROOTED', 'FREE', 'CONICAL', 'FLAT', 'KNOBBED', 'URBAN', 'PINK', 'ABUNDANT', 'NARROW', 'SCATTERED', 'NO', 'BUFF', 'WASTE', 'PARTIAL', 'BLACK', 'FISHY', 'PUNGENT', 'PURPLE', 'PENDANT', 'NONE', 'GRAY', 'LARGE', 'GROOVES', 'SCALY', 'ORANGE', 'MEADOWS', 'WOODS', 'SEVERAL', 'POISONOUS', 'BROAD', 'GREEN', '?', 'SOLITARY', 'FOUL', 'SILKY', 'NUMEROUS', 'CLOSE', 'CONVEX', 'SMOOTH', 'BULBOUS', 'BROWN'}
Exemple: singletons(Base_mushrooms) rend :
[{'ALMOND'}, {'ENLARGING'}, {'CREOSOTE'}, {'GRASSES'}, {'YELLOW'}, {'PATHS'}, {'BELL'}, {'FLARING'}, {'CINNAMON'}, {'CHOCOLATE'}, {'CROWDED'}, {'EVANESCENT'}, {'CLUSTERED'}, {'RED'}, {'BRUISES'}, {'EDIBLE'}, {'CLUB'}, {'SUNKEN'}, {'ATTAC

In [22]:
print(apriori(Base_mushrooms,0.9))

[({'ONE'}, 0.9230038022813688), ({'WHITE'}, 0.9771863117870723), ({'FREE'}, 0.9743346007604563), ({'PARTIAL'}, 1.0), ({'WHITE', 'ONE'}, 0.9001901140684411), ({'FREE', 'ONE'}, 0.9001901140684411), ({'ONE', 'PARTIAL'}, 0.9230038022813688), ({'FREE', 'WHITE'}, 0.9743346007604563), ({'WHITE', 'PARTIAL'}, 0.9771863117870723), ({'FREE', 'PARTIAL'}, 0.9743346007604563), ({'FREE', 'WHITE', 'ONE'}, 0.9001901140684411), ({'WHITE', 'ONE', 'PARTIAL'}, 0.9001901140684411), ({'FREE', 'ONE', 'PARTIAL'}, 0.9001901140684411), ({'FREE', 'WHITE', 'PARTIAL'}, 0.9743346007604563), ({'ONE', 'FREE', 'WHITE', 'PARTIAL'}, 0.9001901140684411)]


In [23]:
print("Résultat du chargement de 'titanic-red.csv', on obtient : ")
Base_titanic = chargeBase('datasets/titanic-red.csv',delimiteur=',')
print(len(Base_titanic),' transactions.')

Résultat du chargement de 'titanic-red.csv', on obtient : 
741  transactions.


In [24]:
# décommenter les lignes suivantes:

print("Pour la BASE Titanic :")
noms_items(Base_titanic)

print("Exemple: singletons(Base_titanic) rend :")
singletons(Base_titanic)

Pour la BASE Titanic :
Exemple: singletons(Base_titanic) rend :


[{'OmahaNE'},
 {'TuxedoParkNY'},
 {'NewYorkNYWashingtonDC'},
 {'RotterdamNetherlands'},
 {'Syria'},
 {'Ireland'},
 {'GlasgowBangorME'},
 {'FinlandMinneapolisMN'},
 {'MexicoCityMexico'},
 {'MerrillWI'},
 {'LosAngelesCA'},
 {'BaselSwitzerland'},
 {'TranvikFinlandNewYork'},
 {'WashingtonDC'},
 {'RuotsinphyhtaaFinlandNewYorkNY'},
 {'Liverpool'},
 {'EnglandDetroitMI'},
 {'KontiolahtiFinlandDetroitMI'},
 {'EastBridgewaterMA'},
 {'AughnacliffCoLongfordIrelandNewYorkNY'},
 {'SwedenWinnipegMN'},
 {'ClevedonEngland'},
 {'alive'},
 {'DuluthMN'},
 {'BirkdaleEnglandClevelandOhio'},
 {'BostonMA'},
 {'MedeltorpSwedenChicagoIL'},
 {'CornwallAkronOH'},
 {'SanFranciscoCA'},
 {'NorwichNewYorkNY'},
 {'RochesterNY'},
 {'BrunswickME'},
 {'LondonEnglandNorfolkVA'},
 {"StAnne'sonSeaLancashire"},
 {'BallydehobCoCorkIrelandNewYorkNY'},
 {'ParisNewYorkNY'},
 {'EnglandBenningtonVT'},
 {'ParisFrance'},
 {'AberdeenPortlandOR'},
 {'YoungstownOH'},
 {'BronxNY'},
 {'HongKongNewYorkNY'},
 {'WestHampsteadLondonNeepawaMB

In [25]:
Lres = apriori(Base_titanic,0.2)

# tri sur les valeurs de supports:
print(sorted(Lres, key=lambda data: data[1]))

# tri sur la taille des itemsets
print(sorted(Lres, key=lambda data: len(data[0])))

for t in sorted(Lres, key=lambda data: data[1],reverse = True):
    if len(t[0])>1:
        print(t)


[({'Southampton', 'alive', 'female'}, 0.203778677462888), ({'2nd', 'male'}, 0.20647773279352227), ({'Southampton', '1st'}, 0.20917678812415655), ({'1st', 'male'}, 0.21997300944669365), ({'1st', 'alive'}, 0.23076923076923078), ({'Cherbourg'}, 0.23751686909581646), ({'Southampton', 'female'}, 0.2483130904183536), ({'3rd'}, 0.27125506072874495), ({'Southampton', 'alive'}, 0.2982456140350877), ({'Southampton', '2nd'}, 0.30634278002699056), ({'alive', 'female'}, 0.32388663967611336), ({'2nd'}, 0.3481781376518219), ({'Southampton', 'dead', 'male'}, 0.3684210526315789), ({'1st'}, 0.3792172739541161), ({'female'}, 0.38866396761133604), ({'Southampton', 'dead'}, 0.41295546558704455), ({'Southampton', 'male'}, 0.46288798920377866), ({'alive'}, 0.4642375168690958), ({'dead', 'male'}, 0.46963562753036436), ({'dead'}, 0.5344129554655871), ({'male'}, 0.6099865047233468), ({'Southampton'}, 0.7112010796221323)]
[({'alive'}, 0.4642375168690958), ({'male'}, 0.6099865047233468), ({'Southampton'}, 0.71120

<font color="RED" size="+1">**[Q]**</font>  Utiliser l'éxécutable <code>apriori</code> pour générer des règles d'association (voir les différentes options en annexe) sur les différentes bases de données fournies. Tester différents seuils (support et confiance). 

In [26]:
!./apriori32 -tr datasets/exemple-1.txt test1.out

./apriori32 - find frequent item sets with the apriori algorithm
version 6.17 (2015.02.25)        (c) 1996-2015   Christian Borgelt
reading datasets/exemple-1.txt ... [5 item(s), 10 transaction(s)] done [0.00s].
building transaction tree ... [13 node(s)] done [0.00s].
checking subsets of size 1 2 3 4 done [0.00s].
writing test1.out ... [9 rule(s)] done [0.00s].


In [27]:
!cat test1.out

d <- e a (10, 100)
d <- e c (10, 100)
d <- e (30, 100)
b <- a c d (20, 100)
c <- a b (50, 80)
b <- a c (40, 100)
b <- a (60, 83.3333)
c <- b (70, 85.7143)
b <- c (70, 85.7143)


In [28]:
!./apriori32 -tr -s70 -m2 -n5 datasets/mushrooms.txt test2.out

./apriori32 - find frequent item sets with the apriori algorithm
version 6.17 (2015.02.25)        (c) 1996-2015   Christian Borgelt
reading datasets/mushrooms.txt ... [70 item(s), 8416 transaction(s)] done [0.01s].
building transaction tree ... [62 node(s)] done [0.00s].
checking subsets of size 1 2 3 4 5 done [0.00s].
writing test2.out ... [102 rule(s)] done [0.00s].


In [29]:
!cat test2.out

CLOSE <- SMOOTH ONE PARTIAL (72.1245, 81.0544)
CLOSE <- SMOOTH ONE (72.1245, 81.0544)
ONE <- SMOOTH FREE WHITE PARTIAL (76.4496, 91.3584)
ONE <- SMOOTH FREE WHITE (76.4496, 91.3584)
FREE <- SMOOTH ONE PARTIAL (72.1245, 96.8369)
ONE <- SMOOTH FREE PARTIAL (76.4496, 91.3584)
FREE <- SMOOTH ONE (72.1245, 96.8369)
ONE <- SMOOTH FREE (76.4496, 91.3584)
WHITE <- SMOOTH ONE PARTIAL (72.1245, 96.8369)
ONE <- SMOOTH WHITE PARTIAL (76.4496, 91.3584)
WHITE <- SMOOTH ONE (72.1245, 96.8369)
ONE <- SMOOTH WHITE (76.4496, 91.3584)
PARTIAL <- SMOOTH ONE (72.1245, 100)
ONE <- SMOOTH PARTIAL (78.731, 91.6088)
ONE <- SMOOTH (78.731, 91.6088)
PARTIAL <- SMOOTH FREE WHITE (76.4496, 100)
WHITE <- SMOOTH FREE PARTIAL (76.4496, 100)
FREE <- SMOOTH WHITE PARTIAL (76.4496, 100)
WHITE <- SMOOTH FREE (76.4496, 100)
FREE <- SMOOTH WHITE (76.4496, 100)
PARTIAL <- SMOOTH FREE (76.4496, 100)
FREE <- SMOOTH PARTIAL (78.731, 97.1023)
FREE <- SMOOTH (78.731, 97.1023)
PARTIAL <- SMOOTH WHITE (76.4496, 100)
WHITE <- SMOOT

In [30]:
!./apriori32 -tr -s20 -m2 -n5 datasets/titanic-red.csv test3.out

./apriori32 - find frequent item sets with the apriori algorithm
version 6.17 (2015.02.25)        (c) 1996-2015   Christian Borgelt
reading datasets/titanic-red.csv ... [378 item(s), 741 transaction(s)] done [0.00s].
building transaction tree ... [48 node(s)] done [0.00s].
checking subsets of size 1 2 3 4 done [0.00s].
writing test3.out ... [7 rule(s)] done [0.00s].


In [32]:
!cat test3.out

dead <- 2nd male (20.6478, 83.6601)
Southampton <- 2nd male (20.6478, 88.2353)
Southampton <- 2nd (34.8178, 87.9845)
alive <- female Southampton (24.8313, 82.0652)
alive <- female (38.8664, 83.3333)
male <- dead Southampton (41.2955, 89.2157)
male <- dead (53.4413, 87.8788)


<font color="RED" size="+1">**[Q]**</font>   Ajouter les deux mesures d'intérêt vues en TD (lift et RR) et afficher leurs valeurs pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.


In [33]:
# lift correspond à l'option -el d'après la documentation fournie + -d pour définir la borne inf de cette mesure

!./apriori32 -tr -s20 -m2 -n5 -el -d150 datasets/titanic-red.csv test4.out

./apriori32 - find frequent item sets with the apriori algorithm
version 6.17 (2015.02.25)        (c) 1996-2015   Christian Borgelt
reading datasets/titanic-red.csv ... [378 item(s), 741 transaction(s)] done [0.00s].
building transaction tree ... [48 node(s)] done [0.00s].
checking subsets of size 1 2 3 4 done [0.00s].
writing test4.out ... [3 rule(s)] done [0.00s].


In [34]:
!cat test4.out

dead <- 2nd male (20.6478, 83.6601)
alive <- female Southampton (24.8313, 82.0652)
alive <- female (38.8664, 83.3333)


In [35]:
# Pour RR, nous allons avoir besoin de l'implémenter en brut :

In [49]:
def read_transacs(file):
    A = []
    B = []
    with open(file, 'r') as source:
        for line in source:
            tmp = line.rstrip().split(" <- ")
            A.append(tmp[0])
            tmp2 = set(tmp[1].split(' ')[:-2])
            B.append(tmp2)
    
    return A,B

In [50]:
A,B = read_transacs('test3.out')

In [51]:
def RR(a,b,A,B): # b -> a transaction entre deux éléments qui sont a dans A et b dans B
    n_ba = 0
    n_b = 0
    n_bbara = 0
    n_bbar = 0
    i = 0
    while i<len(A):
        if B[i] == b:
            n_b += 1
            if A[i] == a:
                n_ba += 1
        else:
            n_bbar += 1
            if A[i] == a:
                n_bbara += 1
                
        i += 1
    
    if n_b != 0 and n_bbara != 0:
        return (n_ba / n_b) * (n_bbar / n_bbara)
    else:
        return "RR non disponible (division par 0)"

In [52]:
i=0
while i<len(A):
    print("RR("+A[i]+" <- "+str(B[i])+") : " + str(RR(A[i],B[i],A,B)))
    i+=1

RR(dead <- {'male', '2nd'}) : RR non disponible (division par 0)
RR(Southampton <- {'male', '2nd'}) : 2.5
RR(Southampton <- {'2nd'}) : 6.0
RR(alive <- {'Southampton', 'female'}) : 6.0
RR(alive <- {'female'}) : 6.0
RR(male <- {'Southampton', 'dead'}) : 6.0
RR(male <- {'dead'}) : 6.0


<font color="RED" size="+1">**[Q]**</font>  Ajouter les deux mesures d'intérêt suivantes et afficher leurs valeurs
  pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.
  
Par exemple:

\begin{align}
  \mbox{Interest}(X \longrightarrow Y) & =  \frac{P(X,Y)}{P(X)}P(Y) \nonumber\\
 \mbox{IS}(X \longrightarrow Y) & =  \frac{P(X,Y)}{\sqrt{P(X)P(Y)}} \nonumber
\end{align}

In [43]:
def interest(a,b,A,B): # b -> a
    n_ba = 0
    n_a = 0
    n_b = 0
    i = 0
    while i<len(A):
        if A[i] == a:
            n_a += 1
        if B[i] == b:
            n_b += 1
        
        if A[i] == a and B[i] == b:
            n_ba += 1
        
                
        i += 1
    
    if n_a != 0:
        return (n_ba / n_b) * n_a
    else:
        return "Interest non disponible (division par 0)"

In [44]:
i=0
while i<len(A):
    print("("+A[i]+" <- "+str(B[i])+") : " + str(interest(A[i],B[i],A,B)))
    i+=1

(dead <- {'male', '2nd'}) : 0.5
(Southampton <- {'male', '2nd'}) : 1.0
(Southampton <- {'2nd'}) : 2.0
(alive <- {'Southampton', 'female'}) : 2.0
(alive <- {'female'}) : 2.0
(male <- {'Southampton', 'dead'}) : 2.0
(male <- {'dead'}) : 2.0


In [55]:
def IS(a,b,A,B): # b -> a
    n_ba = 0
    n_a = 0
    n_b = 0
    i = 0
    while i<len(A):
        if A[i] == a:
            n_a += 1
        if B[i] == b:
            n_b += 1
        
        if A[i] == a and B[i] == b:
            n_ba += 1
        
                
        i += 1
    
    if n_a*n_b != 0:
        return n_ba / (n_b*n_a)**(1/2)
    else:
        return "Interest non disponible (division par 0)"

In [56]:
i=0
while i<len(A):
    print("("+A[i]+" <- "+str(B[i])+") : " + str(IS(A[i],B[i],A,B)))
    i+=1

(dead <- {'male', '2nd'}) : 0.7071067811865475
(Southampton <- {'male', '2nd'}) : 0.5
(Southampton <- {'2nd'}) : 0.7071067811865475
(alive <- {'Southampton', 'female'}) : 0.7071067811865475
(alive <- {'female'}) : 0.7071067811865475
(male <- {'Southampton', 'dead'}) : 0.7071067811865475
(male <- {'dead'}) : 0.7071067811865475


### Utilisation de fp-growth

<font color="RED" size="+1">**[Q]**</font>   Utiliser <code>fpgrowth</code> pour générer des règles d'association sur les différentes
  bases de données. Comparer avec les résultats obtenus dans la section précédente. En  particulier, comparer les temps d'éxecution.

In [58]:
!./fpgrowth32 -tr datasets/exemple-1.txt test1_fp.out
!cat test1_fp.out

./fpgrowth32 - find frequent item sets with the fpgrowth algorithm
version 6.5 (2015.02.25)         (c) 2004-2014   Christian Borgelt
reading datasets/exemple-1.txt ... [5 item(s), 10 transaction(s)] done [0.00s].
filtering, sorting and recoding items ... [5 item(s)] done [0.00s].
sorting and reducing transactions ... [8/10 transaction(s)] done [0.00s].
finding frequent item set(s) ... done [0.00s].
writing test1_fp.out ... [9 rule(s)] done [0.00s].
b <- c (70, 85.7143)
c <- b (70, 85.7143)
b <- a (60, 83.3333)
b <- a c (40, 100)
c <- a b (50, 80)
b <- a d c (20, 100)
d <- e c (10, 100)
d <- e (30, 100)
d <- e a (10, 100)


In [60]:
!./fpgrowth32 -tr -s70 -m2 -n5 datasets/mushrooms.txt test2_fp.out
!cat test2_fp.out

./fpgrowth32 - find frequent item sets with the fpgrowth algorithm
version 6.5 (2015.02.25)         (c) 2004-2014   Christian Borgelt
reading datasets/mushrooms.txt ... [70 item(s), 8416 transaction(s)] done [0.01s].
filtering, sorting and recoding items ... [10 item(s)] done [0.00s].
sorting and reducing transactions ... [37/8416 transaction(s)] done [0.00s].
finding frequent item set(s) ... done [0.00s].
writing test2_fp.out ... [102 rule(s)] done [0.00s].
PARTIAL <- WHITE (97.7186, 100)
WHITE <- PARTIAL (100, 97.7186)
PARTIAL <- FREE (97.4335, 100)
FREE <- PARTIAL (100, 97.4335)
PARTIAL <- FREE WHITE (97.4335, 100)
WHITE <- FREE PARTIAL (97.4335, 100)
FREE <- WHITE PARTIAL (97.7186, 99.7082)
WHITE <- FREE (97.4335, 100)
FREE <- WHITE (97.7186, 99.7082)
PARTIAL <- ONE (92.3004, 100)
ONE <- PARTIAL (100, 92.3004)
PARTIAL <- ONE WHITE (90.019, 100)
WHITE <- ONE PARTIAL (92.3004, 97.5283)
ONE <- WHITE PARTIAL (97.7186, 92.1206)
WHITE <- ONE (92.3004, 97.5283)
ONE <- WHITE (97.7186, 92.1

In [61]:
!./fpgrowth32 -tr -s20 -m2 -n5 datasets/titanic-red.csv test3_fp.out
!cat test3_fp.out

./fpgrowth32 - find frequent item sets with the fpgrowth algorithm
version 6.5 (2015.02.25)         (c) 2004-2014   Christian Borgelt
reading datasets/titanic-red.csv ... [378 item(s), 741 transaction(s)] done [0.00s].
filtering, sorting and recoding items ... [9 item(s)] done [0.00s].
sorting and reducing transactions ... [32/741 transaction(s)] done [0.00s].
finding frequent item set(s) ... done [0.00s].
writing test3_fp.out ... [7 rule(s)] done [0.00s].
male <- dead Southampton (41.2955, 89.2157)
male <- dead (53.4413, 87.8788)
alive <- female Southampton (24.8313, 82.0652)
alive <- female (38.8664, 83.3333)
Southampton <- 2nd (34.8178, 87.9845)
Southampton <- 2nd male (20.6478, 88.2353)
dead <- 2nd male (20.6478, 83.6601)


In [62]:
from time import time

debut = time()
!./apriori32 -trs50c70m2n5 datasets/mushrooms.txt
fin = time()
temps_apriori = fin - debut

debut = time()
!./fpgrowth32 -trs50c70m2n5 datasets/mushrooms.txt
fin = time()
temps_fpgrowth = fin - debut

if temps_fpgrowth > temps_apriori:
    print("FPGrowth est plus rapide que Apriori :", temps_fpgrowth, "s contre", temps_apriori, "s")
elif temps_fpgrowth < temps_apriori:
    print("Apriori est plus rapide que FPGrowth :", temps_apriori, "s contre", temps_fpgrowth, "s")
else:
    print("Les deux algos sont équivalents en temmps d'exécution ("+ str(temps_fpgrowth) + "s chacun.")

./apriori32 - find frequent item sets with the apriori algorithm
version 6.17 (2015.02.25)        (c) 1996-2015   Christian Borgelt
reading datasets/mushrooms.txt ... [70 item(s), 8416 transaction(s)] done [0.01s].
building transaction tree ... [1688 node(s)] done [0.00s].
checking subsets of size 1 2 3 4 5 done [0.00s].
writing <null> ... [594 rule(s)] done [0.00s].
./fpgrowth32 - find frequent item sets with the fpgrowth algorithm
version 6.5 (2015.02.25)         (c) 2004-2014   Christian Borgelt
reading datasets/mushrooms.txt ... [70 item(s), 8416 transaction(s)] done [0.02s].
filtering, sorting and recoding items ... [26 item(s)] done [0.00s].
sorting and reducing transactions ... [675/8416 transaction(s)] done [0.00s].
finding frequent item set(s) ... done [0.00s].
writing <null> ... [594 rule(s)] done [0.00s].
FPGrowth est plus rapide que Apriori : 0.15517830848693848 s contre 0.12295222282409668 s


<font color="RED" size="+1">**[Q]**</font>  En essayant différentes valeurs de seuil, tester la génération de règles d'association intéressantes sur les bases de données fournies.

## Annexes: utilisation des exécutables *apriori* et *fpgrowth*

La documentation complète de ces programmes est disponible aux URL suivantes:
- pour *apriori*: https://www.borgelt.net/doc/apriori/apriori.html
- pour *fp-growth*: https://www.borgelt.net/doc/fpgrowth/fpgrowth.html


Leur format d'utilisation général est:

        ./<programme> [options] infile [outfile]

(où <<code>programme</code>> est soit *apriori*, soit *fpgrowth*).

Les options de base de ces 2 programmes sont:
- sans argument: génération des itemsets fréquents (argument {<code>-ts</code> activé par défaut)
- *tr*: pour obtenir les règles d'associations
- *s*: pour fournir une valeur minimale de support. Le support est ici donné en valeur absolue ($n_{AB}$) et non pas en valeur relative ($\frac{n_{AB}}{n}$).
- *m*: pour fournir un nombre minimum d'items dans un itemset
- *n*: pour fournir un nombre maximum d'items dans un itemset


Par exemple (commandes lancées dans le répertoire père du répertoire *datasets/*):

        ./apriori -trs50m2n5 datasets/exemple-1.txt fichier-resultat.out  
        ./fpgrowth -trs50m2n5 datasets/mushrooms.txt fichier-resultat.out  
